### Ideas on generating question-answer pairs over a specific document that can be used to build the Knowledge base

#### Set the Environment Variable

In [9]:
# Import Python libraries
import os
import openai
from Utilities.envVars import *
from openai import OpenAI, AzureOpenAI, AsyncAzureOpenAI

# Set Search Service endpoint, index name, and API key from environment variables
indexName = SearchIndex

# # Set OpenAI API key and endpoint
# openai.api_type = "azure"
# openai.api_version = OpenAiVersion
# openai_api_key = OpenAiKey
# assert openai_api_key, "ERROR: Azure OpenAI Key is missing"
# openai.api_key = openai_api_key
# openAiEndPoint = f"{OpenAiEndPoint}"
# openai.api_base = openAiEndPoint

azure_endpoint  = f"{OpenAiEndPoint}"
api_key = OpenAiKey
api_version = OpenAiVersion

client = AzureOpenAI(
    api_key=api_key,  
    api_version=api_version,
    #base_url=f"{os.getenv('OpenAiWestUsEp')}openai/deployments/{os.getenv('OpenAiGpt4v')}/extensions",
    azure_endpoint=azure_endpoint,
)

#### Import Required Library

In [3]:
# Import required libraries
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import (
    PDFMinerLoader,
    UnstructuredFileLoader,
)
from langchain.chat_models import ChatOpenAI
from langchain_openai import AzureChatOpenAI
from langchain.chains import QAGenerationChain

In [4]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.chat_models import AzureChatOpenAI, ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
from IPython.display import display, HTML
from langchain_openai import AzureOpenAIEmbeddings

embeddingModelType = "azureopenai"
temperature = 0.3
tokenLength = 500

if (embeddingModelType == 'azureopenai'):
        llm = AzureChatOpenAI(
                azure_endpoint=OpenAiEndPoint,
                api_version=OpenAiVersion,
                azure_deployment=OpenAiChat,
                temperature=temperature,
                api_key=OpenAiKey,
                openai_api_type="azure",
                max_tokens=tokenLength)

        embeddings = AzureOpenAIEmbeddings(azure_endpoint=OpenAiEndPoint, azure_deployment=OpenAiEmbedding, api_key=OpenAiKey, openai_api_type="azure")
        logging.info("LLM Setup done")
elif embeddingModelType == "openai":
        openai.api_type = "open_ai"
        openai.api_base = "https://api.openai.com/v1"
        openai.api_version = '2020-11-07' 
        openai.api_key = OpenAiApiKey
        llm = ChatOpenAI(temperature=temperature,
        openai_api_key=OpenAiApiKey,
        model_name="gpt-3.5-turbo",
        max_tokens=tokenLength)
        embeddings = OpenAIEmbeddings(openai_api_key=OpenAiApiKey)

c:\Users\astalati\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.azure_openai.AzureChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureChatOpenAI`.
  warn_deprecated(


In [5]:
# Set the file name and the namespace for the index
fileName = "Fabric Get Started.pdf"
fabricGetStartedPath = "Data/PDF/" + fileName
# Load the PDF with Document Loader available from Langchain
loader = PDFMinerLoader(fabricGetStartedPath)
rawDocs = loader.load()
# Set the source 
for doc in rawDocs:
    doc.metadata['source'] = fabricGetStartedPath

textSplitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=0)
docs = textSplitter.split_documents(rawDocs)

In [6]:
chain = QAGenerationChain.from_llm(llm)

In [7]:
qa = chain.run(docs[55].page_content)

c:\Users\astalati\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [8]:
qa

[{'question': "What does it mean if you see a 'Make discoverable' checkbox when promoting a Power BI dataset?",
  'answer': "It means you can make it possible for users who don't have access to the dataset to find it."}]